# How do open source projects register releases?

In [2]:
import datetime
import json

# %matplotlib inline
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
pd.options.display.max_rows = 1000

from scipy.stats import wilcoxon
from scipy.stats import ranksums

In [3]:
import os
import sys

releasy_module = os.path.abspath(os.path.join('..','..','..','dev','releasy'))
if releasy_module not in sys.path:
    sys.path.insert(0, releasy_module)
    
import releasy

In [67]:
repo_path = os.path.join('..','..','..','repos')
projects = pd.read_pickle("projects.zip")

In [86]:
project_df = pd.DataFrame(projects.index).set_index('name')
project_df['path'] = repo_path + project_df.index

In [92]:
for prj in project_df.itertuples():
    releasy.mine(prj.path)
    project_df.loc[prj.Index, "data"] = prj.path
    

AttributeError: module 'releasy' has no attribute 'mine'

In [91]:
project_df

,path,data
name,,
freeCodeCamp/freeCodeCamp,../../../reposfreeCodeCamp/freeCodeCamp,../../../reposfreeCodeCamp/freeCodeCamp
vuejs/vue,../../../reposvuejs/vue,../../../reposvuejs/vue
facebook/react,../../../reposfacebook/react,../../../reposfacebook/react
twbs/bootstrap,../../../repostwbs/bootstrap,../../../repostwbs/bootstrap
facebook/react-native,../../../reposfacebook/react-native,../../../reposfacebook/react-native
facebook/create-react-app,../../../reposfacebook/create-react-app,../../../reposfacebook/create-react-app
axios/axios,../../../reposaxios/axios,../../../reposaxios/axios
nodejs/node,../../../reposnodejs/node,../../../reposnodejs/node
FortAwesome/Font-Awesome,../../../reposFortAwesome/Font-Awesome,../../../reposFortAwesome/Font-Awesome


In [5]:
count = 1
projects["data"] = None
for name, project in projects[(projects.data.isnull())].iterrows():
    print(f"{datetime.datetime.now()} - {count:3} - Processing {name}")
    project_group, project_name = name.split("/") 
    path = os.path.join(repo_path, project_group, f"{project_name}.git")
    projects.loc[name, "path"] = path    
    
    mine_params = {
        "track_base_release": False
    }
    if name in release_mine_params:
        mine_params.update(release_mine_params[name])
    
    miner = releasy_miner.Miner(name=name,vcs=releasy_git.GitVcs(path), **mine_params)
    project = miner.mine_commits()
    projects.loc[name, "data"] = project
    count += 1
print(f"{datetime.datetime.now()} - Ended")

,description,stars,url,git_url,language,created_at,active
name,,,,,,,
freeCodeCamp/freeCodeCamp,The https://www.freeCodeCamp.org open source c...,308061.0,https://api.github.com/repos/freeCodeCamp/free...,git://github.com/freeCodeCamp/freeCodeCamp.git,JavaScript,2014-12-24 17:49:19,True
vuejs/vue,"🖖 Vue.js is a progressive, incrementally-adopt...",155509.0,https://api.github.com/repos/vuejs/vue,git://github.com/vuejs/vue.git,JavaScript,2013-07-29 03:24:51,True
facebook/react,"A declarative, efficient, and flexible JavaScr...",142182.0,https://api.github.com/repos/facebook/react,git://github.com/facebook/react.git,JavaScript,2013-05-24 16:15:54,True
twbs/bootstrap,"The most popular HTML, CSS, and JavaScript fra...",138078.0,https://api.github.com/repos/twbs/bootstrap,git://github.com/twbs/bootstrap.git,JavaScript,2011-07-29 21:19:00,True
facebook/react-native,A framework for building native apps with React.,84064.0,https://api.github.com/repos/facebook/react-na...,git://github.com/facebook/react-native.git,JavaScript,2015-01-09 18:10:16,True
facebook/create-react-app,Set up a modern web app by running one command.,75160.0,https://api.github.com/repos/facebook/create-r...,git://github.com/facebook/create-react-app.git,JavaScript,2016-07-17 14:55:11,True
axios/axios,Promise based HTTP client for the browser and ...,68592.0,https://api.github.com/repos/axios/axios,git://github.com/axios/axios.git,JavaScript,2014-08-18 22:30:27,True
nodejs/node,Node.js JavaScript runtime :sparkles::turtle::...,67073.0,https://api.github.com/repos/nodejs/node,git://github.com/nodejs/node.git,JavaScript,2014-11-26 19:57:11,True
FortAwesome/Font-Awesome,"The iconic SVG, font, and CSS toolkit",61836.0,https://api.github.com/repos/FortAwesome/Font-...,git://github.com/FortAwesome/Font-Awesome.git,JavaScript,2012-02-17 14:19:43,True
